In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages --------------------------------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ------------------------------------------------------------------------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [3]:
# Define constants
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

event_code <- list("Alive" = 0, "Dead" = 1)
# covariate_cols_no_figo <- c("age_at_diagnosis", "race", "ethnicity")
# covariate_cols <- c("figo_stage", covariate_cols_no_figo)
covariate_cols <- c("figo_stage")
dep_cols <- c("vital_status", "survival_time")

In [5]:
survival_dfs <- list()
for (dset_idx in 1:3) {
    survival_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/survival_data.tsv")
    survival_df <- load_survival_df(survival_path, event_code)

    # Load and filter survival data
    survival_dfs[[unified_dsets[dset_idx]]] <- survival_df %>%
        decode_figo_stage(to = "c") %>%
        dplyr::select(sample_name, figo_stage) %>% # make sure using same samples as classification models
        dplyr::filter(rowSums(is.na(.)) == 0)

}



-- Column specification --------------------------------------------------------------------------------------------------------------------------
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_double(),
  weight = col_double(),
  bmi = col_double(),
  race = col_character(),
  ethnicity = col_character()
)



-- Column specification --------------------------------------------------------------------------------------------------------------------------
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_doubl

In [13]:
figo_stages <- function(df) {
    df %>%
        group_by(figo_stage) %>%
        summarize(n = n()) %>%
        mutate(pct = round(n / sum(n), 2))
}


In [15]:
figo_stages(survival_dfs[[unified_dsets[1]]])
figo_stages(survival_dfs[[unified_dsets[2]]])
figo_stages(survival_dfs[[unified_dsets[3]]])

,figo_stage,n,pct
,<chr>,<int>,<dbl>
1,figo_stage_1,135,0.53
2,figo_stage_2,61,0.24
3,figo_stage_3,40,0.16
4,figo_stage_4,19,0.07


,figo_stage,n,pct
,<chr>,<int>,<dbl>
1,figo_stage_1,17,0.37
2,figo_stage_2,5,0.11
3,figo_stage_3,18,0.39
4,figo_stage_4,6,0.13


,figo_stage,n,pct
,<chr>,<int>,<dbl>
1,figo_stage_1,75,0.54
2,figo_stage_2,19,0.14
3,figo_stage_3,39,0.28
4,figo_stage_4,7,0.05
